In [1]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV, cross_validate
from sklearn.model_selection import train_test_split


from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
from aequitas.plotting import Plot

%matplotlib inline

## Incorporating fairness into cross validation

Read in data 

In [16]:
### load data
data = pd.read_csv("../data/broward_data.csv")

X = data.loc[:,:'five_year']
Y = data['recid_two_year']

In [25]:
data.columns

Index(['person_id', 'screening_date', 'sex', 'p_current_age',
       'p_age_first_offense', 'p_charge', 'p_jail30', 'p_prison',
       'p_probation', 'p_juv_fel_count', 'p_felprop_violarrest',
       'p_murder_arrest', 'p_felassault_arrest', 'p_misdemassault_arrest',
       'p_famviol_arrest', 'p_sex_arrest', 'p_weapons_arrest',
       'fail_appear_two_yr', 'fail_appear_two_plus', 'current_violent',
       'current_violent20', 'pending_charge', 'prior_conviction_F',
       'prior_conviction_M', 'violent_conviction', 'total_convictions',
       'p_arrest', 'p_property', 'p_traffic', 'p_drug', 'p_dui', 'p_domestic',
       'p_stalking', 'p_voyeurism', 'p_fraud', 'p_stealing', 'p_trespass',
       'years_since_last_crime', 'six_month', 'one_year', 'three_year',
       'five_year', 'recid_two_year', 'recid_six_month', 'recid_drug',
       'recid_property', 'recid_M', 'recid_F', 'recid_violent'],
      dtype='object')

In [21]:
### Implementing fairness as scoring method 


def compute_fairness(df: pd.DataFrame, 
                     decoders: dict, 
                     sensitive_attrs: list,
                     ref_groups_dict: dict) -> pd.DataFrame:
    """
    decoders: dictionary of dictionary of decoders 
    """
    df = df.rename({"person_id": "entity_id"}, axis="columns")
    # decode numeric encodings for cat var
    for decoder_name, decoder_dict in decoders.items():
        df = df.replace({decoder_name: decoder_dict})
    
    g = Group()
    xtab, _ = g.get_crosstabs(df, attr_cols=sensitive_attrs)
    # compute bias 
    b = Bias()
    bdf = b.get_disparity_predefined_groups(xtab, 
                                            original_df=df, 
                                            ref_groups_dict=ref_groups_dict, 
                                            alpha=0.05, 
#                                           check_significance=True, 
#                                           mask_significance=True
                                            )
    f = Fairness()
    fdf = f.get_group_value_fairness(bdf)

    # list results of fairness analysis
    parity_determinations = f.list_parities(fdf)
    
    absolute_metrics = g.list_absolute_metrics(xtab)
    return fdf[['attribute_name', 'attribute_value'] + absolute_metrics + b.list_disparities(fdf) + parity_determinations].style

In [22]:
# prepare data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.3, shuffle = True, random_state = 5829)

clf = LogisticRegression(class_weight = 'balanced', solver='liblinear', random_state=0)
clf.fit(X_train.drop(['person_id', 'screening_date'], axis=1),
        Y_train)
preds = clf.predict(X_test.drop(['person_id', 'screening_date'], axis=1))
X_test.loc[:,"score"] = preds
X_test.loc[:,"label_value"] = Y_test

df = X_test[["person_id", "screening_date", "sex", "score", "label_value"]]
df = df.rename({"person_id": "entity_id"}, axis="columns")


decoders = {"sex": {0: "male",
                    1: "female"}
           }

sensitive_attrs = ['sex'] # race 

ref_groups_dict = {'sex':'male'}
res = compute_fairness(df, 
                       decoders=decoders, 
                       sensitive_attrs=sensitive_attrs,
                       ref_groups_dict=ref_groups_dict)

res

C:\Users\Caroline Wang\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Caroline Wang\Anaconda3\lib\site-packages\pandas\core\indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


model_id, score_thresholds 1 {'rank_abs': [307]}
get_disparity_predefined_group()


,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,ppr,pprev,prev,ppr_disparity,pprev_disparity,precision_disparity,fdr_disparity,for_disparity,fpr_disparity,fnr_disparity,tpr_disparity,tnr_disparity,npv_disparity,TNR Parity,FNR Parity,Statistical Parity,FPR Parity,Supervised Fairness,TypeII Parity,FDR Parity,TypeI Parity,Impact Parity,Precision Parity,NPV Parity,FOR Parity,Equalized Odds,TPR Parity,Unsupervised Fairness
0,sex,female,0.470588,0.9,0.285714,0.238095,0.1,0.529412,0.714286,0.761905,0.0684039,0.25,0.404762,0.0734266,0.439685,1.28938,0.582011,0.925373,0.228205,1.86479,0.657153,1.602,1.03333,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False
1,sex,male,0.716102,0.561798,0.308756,0.409091,0.438202,0.283898,0.691244,0.590909,0.931596,0.568588,0.469185,1,1,1,1,1,1,1,1,1,1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


### Create Custom Scorer

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring

In [7]:
def my_custom_loss_func(y_true, y_pred):
    diff = np.abs(y_true - y_pred).max()
    return np.log1p(diff)

 # score will negate the return value of my_custom_loss_func,
 # which will be np.log(2), 0.693, given the values for X
 # and y defined below.
score = make_scorer(my_custom_loss_func, greater_is_better=False)
X = [[1], [1]]
y = [0, 1]
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent', random_state=0)
clf = clf.fit(X, y)
print(my_custom_loss_func(y, clf.predict(X)) )

print(score(clf, X, y))


0.6931471805599453
-0.6931471805599453


### Adapt current logistic prediction method 

#### Original

In [17]:
def Logistic(x, y, C,seed):
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import KFold, GridSearchCV, cross_validate
    
    ## cross validation set up
    inner_cv = KFold(n_splits=5,shuffle=True,random_state=seed)
    outer_cv = KFold(n_splits=5,shuffle=True,random_state=seed)
    
    ### model & parameters
    lr = LogisticRegression(class_weight = 'balanced', solver='liblinear', random_state=seed)
    c_grid = {"C": C}
    
    ### nested cross validation
    clf = GridSearchCV(estimator=lr, param_grid=c_grid, scoring='roc_auc',cv=inner_cv, return_train_score=True)
    nested_score = cross_validate(clf, X=x, y=y, scoring='roc_auc', cv=outer_cv, return_train_score=True)
    train_score, test_score = nested_score['train_score'], nested_score['test_score']
    
    return train_score, test_score

In [18]:
c = [1e-5, 1e-4, 1e-3]

logistic_train, logistic_test = Logistic(X.drop(['person_id', 'screening_date'], axis=1),
                                         Y,
                                         c,
                                         816)
res = [np.mean(logistic_train) - np.mean(logistic_test), np.mean(logistic_test)]
res

[0.011141768594555845, 0.6651818167782302]

#### New

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV, cross_validate

def Logistic(X, y, C,seed):
    """
    X: pandas dataframe including ID columns
    Y:
    """
    X_no_id = X.drop(["person_id", "screening_date"], axis=1)
    # cross validation set up
    inner_cv = KFold(n_splits=5,shuffle=True,random_state=seed)
    outer_cv = KFold(n_splits=5,shuffle=True,random_state=seed)
    
    # model & parameters
    lr = LogisticRegression(class_weight = 'balanced', solver='liblinear', random_state=seed)
    c_grid = {"C": C}
    
    # nested cross validation
    clf = GridSearchCV(estimator=lr, param_grid=c_grid, scoring='roc_auc',cv=inner_cv, return_train_score=True)
    nested_score = cross_validate(clf, X=X_no_id, y=y, scoring='roc_auc', cv=outer_cv, return_train_score=True)
    train_score, test_score = nested_score['train_score'], nested_score['test_score']
    
    
    # prepare for fairness analysis
#     preds = clf.predict(X_test.drop(['person_id', 'screening_date'], axis=1))
#     X_test.loc[:,"score"] = preds
#     X_test.loc[:,"label_value"] = Y_test
#     df = X_test[["person_id", "screening_date", "sex", "score", "label_value"]]
#     df["entity_id"] = " ".join(df["person_id"], df["entity_id"])
#     df.drop(["person_id", "screening_date"])
    return train_score, test_score

In [24]:
X.columns

Index(['person_id', 'screening_date', 'sex', 'p_current_age',
       'p_age_first_offense', 'p_charge', 'p_jail30', 'p_prison',
       'p_probation', 'p_juv_fel_count', 'p_felprop_violarrest',
       'p_murder_arrest', 'p_felassault_arrest', 'p_misdemassault_arrest',
       'p_famviol_arrest', 'p_sex_arrest', 'p_weapons_arrest',
       'fail_appear_two_yr', 'fail_appear_two_plus', 'current_violent',
       'current_violent20', 'pending_charge', 'prior_conviction_F',
       'prior_conviction_M', 'violent_conviction', 'total_convictions',
       'p_arrest', 'p_property', 'p_traffic', 'p_drug', 'p_dui', 'p_domestic',
       'p_stalking', 'p_voyeurism', 'p_fraud', 'p_stealing', 'p_trespass',
       'years_since_last_crime', 'six_month', 'one_year', 'three_year',
       'five_year'],
      dtype='object')